In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

# from torchtext.datasets import TranslationDataset, Multi30k
# from torchtext.data import Field, BucketIterator

# import spacy

import pickle
import random
import math
import os
import time
import nltk

from sacremoses import MosesTokenizer
usingBPE = False

In [2]:
train_en = []
train_de = []
with open("data/train.en", "r", encoding="utf8") as f:
    train_en = f.readlines()
with open("data/train.de", "r", encoding="utf8") as f:
    train_de = f.readlines()
with open("data/dev.en", "r", encoding="utf8") as f:
    dev_en = f.readlines()
with open("data/dev.de", "r", encoding="utf8") as f:
    dev_de = f.readlines()
with open("data/test.de", "r", encoding="utf8") as f:
    test_de = f.readlines()

print(len(train_de))
print(len(train_en))

196884
196884


In [3]:
# # Get rid of any white space or \n's
# nltk.download('punkt')
# start = "<sos>"
# end = "<eos>"
# pad = "<pad>"
# unk = "<unk>"

## Use the Moses Tokenizer to tokenize the data

In [3]:
start = "<sos>"
end = "<eos>"
pad = "<pad>"
unk = "<unk>"

mt_en = MosesTokenizer(lang='en')
mt_de = MosesTokenizer(lang='de')

for i, sentence in enumerate(train_en):
    tokenized_sentence = mt_en.tokenize(sentence)
#     tokenized_sentence = [start] + tokenized_sentence + [end]
    train_en[i] = tokenized_sentence
    
for i, sentence in enumerate(dev_en):
    tokenized_sentence = mt_en.tokenize(sentence)
#     tokenized_sentence = [start] + tokenized_sentence + [end]
    dev_en[i] = tokenized_sentence
    
for i, sentence in enumerate(train_de):
    tokenized_sentence = mt_de.tokenize(sentence)
#     tokenized_sentence = [start] + tokenized_sentence + [end]
    train_de[i] = tokenized_sentence
    
for i, sentence in enumerate(dev_de):
    tokenized_sentence = mt_de.tokenize(sentence)
#     tokenized_sentence = [start] + tokenized_sentence + [end]
    dev_de[i] = tokenized_sentence
    
for i, sentence in enumerate(test_de):
    tokenized_sentence = mt_de.tokenize(sentence)
#     tokenized_sentence = [start] + tokenized_sentence + [end]
    test_de[i] = tokenized_sentence

## Write the tokenized data to a file to be processed by byte pair encoding

In [4]:
with open('data/tokenized/train.tk.en', 'w', encoding="utf8") as f:
    for sent in train_en:
        x = " ".join(sent)
        f.write(x + '\n')
        
with open('data/tokenized/dev.tk.en', 'w', encoding="utf8") as f:
    for sent in dev_en:
        x = " ".join(sent)
        f.write(x + '\n')
        
with open('data/tokenized/train.tk.de', 'w', encoding="utf8") as f:
    for sent in train_de:
        x = " ".join(sent)
        f.write(x + '\n')
        
with open('data/tokenized/dev.tk.de', 'w', encoding="utf8") as f:
    for sent in dev_de:
        x = " ".join(sent)
        f.write(x + '\n')
        
with open('data/tokenized/test.tk.de', 'w', encoding="utf8") as f:
    for sent in test_de:
        x = " ".join(sent)
        f.write(x + '\n')
        
del train_en, train_de, dev_en, dev_de, test_de

### Learn byte pair encoding on the training set

In [5]:
! cd data && subword-nmt learn-joint-bpe-and-vocab --input tokenized/train.tk.de tokenized/train.tk.en -s 35000 -o bpe/codes.txt --write-vocabulary bpe/vocab.de bpe/vocab.en

### Apply byte pair encoding with vocabulary filters to all datasets

In [6]:
! cd data && subword-nmt apply-bpe -c bpe/codes.txt --vocabulary bpe/vocab.de --vocabulary-threshold 50 < tokenized/train.tk.de > bpe/train.BPE.de

! cd data && subword-nmt apply-bpe -c bpe/codes.txt --vocabulary bpe/vocab.en --vocabulary-threshold 50 < tokenized/train.tk.en > bpe/train.BPE.en
 
! cd data && subword-nmt apply-bpe -c bpe/codes.txt --vocabulary bpe/vocab.en --vocabulary-threshold 50 < tokenized/dev.tk.en > bpe/dev.BPE.en
 
! cd data && subword-nmt apply-bpe -c bpe/codes.txt --vocabulary bpe/vocab.de --vocabulary-threshold 50 < tokenized/dev.tk.de > bpe/dev.BPE.de

! cd data && subword-nmt apply-bpe -c bpe/codes.txt --vocabulary bpe/vocab.de --vocabulary-threshold 50 < tokenized/test.tk.de > bpe/test.BPE.de

## Read in the tokenized byte pair encoded data

In [7]:
train_en = []
train_de = []
dev_en = []
dev_de = []
test_de = []

with open('data/bpe/train.BPE.en','r', encoding="utf8") as f:
    for line in f:
        sent = []
        for word in line.split():
            sent.append(word)
        train_en.append(sent)
        
with open('data/bpe/dev.BPE.en','r', encoding="utf8") as f:
    for line in f:
        sent = []
        for word in line.split():
            sent.append(word)
        dev_en.append(sent)
        
with open('data/bpe/train.BPE.de','r', encoding="utf8") as f:
    for line in f:
        sent = []
        for word in line.split():
            sent.append(word)
        train_de.append(sent)
        
with open('data/bpe/dev.BPE.de','r', encoding="utf8") as f:
    for line in f:
        sent = []
        for word in line.split():
            sent.append(word)
        dev_de.append(sent)
        
with open('data/bpe/test.BPE.de','r', encoding="utf8") as f:
    for line in f:
        sent = []
        for word in line.split():
            sent.append(word)
        test_de.append(sent)
        
usingBPE = True

## Use the natural language toolkit to tokenize the words

In [5]:
# # DEPRECATED

# start = "<sos>"
# end = "<eos>"
# pad = "<pad>"
# max_length_en = -1
# for i, sentence in enumerate(train_en):
#     sentence = nltk.word_tokenize(sentence.lower())
#     sentence = [start] + sentence
#     sentence.append(end)
#     max_length_en = len(sentence) if len(sentence) > max_length_en else max_length_en
#     train_en[i] = sentence
    

# max_length_de = -1
# for i, sentence in enumerate(train_de):
#     sentence = nltk.word_tokenize(sentence.lower())
#     sentence = [start] + sentence
#     sentence.append(end)
#     max_length_de = len(sentence) if len(sentence) > max_length_de else max_length_de
#     train_de[i] = sentence
    
    
# max_length_en_dev = -1
# for i, sentence in enumerate(dev_en):
#     sentence = nltk.word_tokenize(sentence.lower())
#     sentence = [start] + sentence
#     sentence.append(end)
#     max_length_en_dev = len(sentence) if len(sentence) > max_length_en_dev else max_length_en_dev
#     dev_en[i] = sentence

# max_length_de_dev = -1
# for i, sentence in enumerate(dev_de):
#     sentence = nltk.word_tokenize(sentence.lower())
#     sentence = [start] + sentence
#     sentence.append(end)
#     max_length_de_dev = len(sentence) if len(sentence) > max_length_de_dev else max_length_de_dev
#     dev_de[i] = sentence

# =============== WARNING ===============

## The following code cell deletes a number of sentences from the training set in order to get the dimensionality of the sentences below a certain threshold. The threshold is the value in the inequality of the while loop

### Removing the longest sentences from the training dataset to decrease the dimensionality of all the sentences.

In [5]:
# DEPRECATED METHOD

# max_idx, en_max_val = max(enumerate(train_en), key=lambda x: len(x[1]))
# _, de_max_val = max(enumerate(train_de), key=lambda x: len(x[1]))

# del_count = 0
# print(max_idx, len(en_max_val), len(de_max_val))

# while len(en_max_val) > 200:
#     del train_en[max_idx]
#     del train_de[max_idx]
    
#     max_idx, en_max_val = max(enumerate(train_en), key=lambda x: len(x[1]))
#     _, de_max_val = max(enumerate(train_de), key=lambda x: len(x[1]))
    
#     print(max_idx, len(en_max_val), len(de_max_val))
    
#     del_count += 1
    
# max_length_en = len(en_max_val)
# max_length_de = len(de_max_val)

In [8]:
# print("Deleted:", del_count)

print("Number of Sentences (training):", len(train_en))
print("Number of Sentences (dev):", len(dev_en))

print("Max length English (training):", max([len(sent) for sent in train_en]) )
print("Max length German (training):", max([len(sent) for sent in train_de]))

print("Max length English (dev):", max([len(sent) for sent in dev_en]))
print("Max length German (dev):", max([len(sent) for sent in dev_de]))
print("Max length German (test):", max([len(sent) for sent in test_de]))

print("")

print(f"Average number of tokens English (training): { sum([len(sent) for sent in train_en])/len(train_en):.2f} ")
print(f"Average number of tokens German (training): { sum([len(sent) for sent in train_de])/len(train_de):.2f} ") 

print(f"Average number of tokens English (dev): { sum([len(sent) for sent in dev_en])/len(dev_en):.2f} ")
print(f"Average number of tokens German (dev): { sum([len(sent) for sent in dev_de])/len(dev_de):.2f} ") 

print(f"Average number of tokens Test Set: { sum([len(sent) for sent in test_de])/len(test_de):.2f} ")


Number of Sentences (training): 196884
Number of Sentences (dev): 7883
Max length English (training): 843
Max length German (training): 858
Max length English (dev): 224
Max length German (dev): 260

Average number of tokens English (training): 23.57 
Average number of tokens German (training): 24.32 
Average number of tokens English (dev): 23.18 
Average number of tokens German (dev): 24.23 
Average number of tokens Test Set: 21.59 


## Choose how large you want the training and validation sets to be
## Don't run this cell if you want to use the entire dataset

In [6]:
TRAIN_SIZE = 100000 # Number from 1 to 196884
VAL_SIZE = 3500 # Number from 1 to 7883

idxs_train = list(range(len(train_en)))
random.shuffle(idxs_train)
idxs_train = idxs_train[0:TRAIN_SIZE]

train_en = [train_en[i] for i in idxs_train]
train_de = [train_de[i] for i in idxs_train]

idxs_val = list(range(len(dev_en)))
random.shuffle(idxs_val)
idxs_val = idxs_val[0:VAL_SIZE]

dev_en = [dev_en[i] for i in idxs_val]
dev_de = [dev_de[i] for i in idxs_val]

## Create a shared vocabulary for both languages

In [9]:
vocab ={}

vocab['<pad>'] = 0
vocab['<unk>'] = 1
vocab['<sos>'] = 2
vocab['<eos>'] = 3
en_inputs = []
de_inputs = []

en_val = []
de_val = []

de_test = []

for sent in train_en:
    idxes = []
    for w in sent:
        if w not in vocab:
            vocab[w] = len(vocab)
        idxes.append(vocab[w])
    en_inputs.append(idxes)
    
for sent in dev_en:
    idxes = []
    for w in sent:
        if w not in vocab:
            vocab[w] = len(vocab)
        idxes.append(vocab[w])
    en_val.append(idxes)
    
for sent in train_de:
    idxes = []
    for w in sent:
        if w not in vocab:
            vocab[w] = len(vocab)
        idxes.append(vocab[w])
    de_inputs.append(idxes)
    
for sent in dev_de:
    idxes = []
    for w in sent:
        if w not in vocab:
            vocab[w] = len(vocab)
        idxes.append(vocab[w])
    de_val.append(idxes)
    
for sent in test_de:
    idxes = []
    for w in sent:
        if w not in vocab:
            vocab[w] = len(vocab)
        idxes.append(vocab[w])
    de_test.append(idxes)
    
print(len(vocab))

12388


## The following cell gets the frequency of words in the dataset

In [10]:
if usingBPE:
    print("using BPE data, skipping this cell")
    pass
else:
    en_freq = {'<pad>':100, # there wont be any padding til later, this is just to
               '<unk>':2,   # avoid removing the padding token
               '<sos>':0,
               '<eos>':0}

    de_freq = {'<pad>':100, # there wont be any padding til later, this is just to
               '<unk>':2,   # avoid removing the padding token
               '<sos>':0,
               '<eos>':0}

    for sent in train_en:
        for w in sent:
            if w not in en_freq:
                en_freq[w] = 1
            else:
                en_freq[w] += 1

    for sent in train_de:
        for w in sent:
            if w not in de_freq:
                de_freq[w] = 1
            else:
                de_freq[w] += 1

    for sent in dev_en:
        for w in sent:
            if w not in en_freq:
                en_freq[w] = 1
            else:
                en_freq[w] += 1

    for sent in dev_de:
        for w in sent:
            if w not in de_freq:
                de_freq[w] = 1
            else:
                de_freq[w] += 1


using BPE data, skipping this cell


## Make a list of uncommon words

In [11]:
if usingBPE:
    print("using BPE data, skipping this cell")
    pass
else:
    en_uncommon = {}
    de_uncommon = {}

    for word, freq in en_freq.items():
        if freq < 2:
            en_uncommon[word] = len(en_uncommon)

    for word, freq in de_freq.items():
        if freq < 2:
            de_uncommon[word] = len(de_uncommon)

    if '<unk>' in en_uncommon:
        print("deleted")
        del en_uncommon['<unk>']

    if '<unk>' in de_uncommon:
        print("deleted")
        del de_uncommon['<unk>']

using BPE data, skipping this cell


## Replace the uncommon words with unknown tokens

In [12]:
if usingBPE:
    print("using BPE data, skipping this cell")
    pass
else:
    en_vocab = {}
    de_vocab = {}
    en_vocab['<pad>'] = 0
    en_vocab['<unk>'] = 1 
    en_vocab['<sos>'] = 2
    en_vocab['<eos>'] = 3

    de_vocab['<pad>'] = 0
    de_vocab['<unk>'] = 1
    de_vocab['<sos>'] = 2
    de_vocab['<eos>'] = 3

    en_inputs = []
    de_inputs = []

    en_val = []
    de_val = []

    for sent in train_en:
        en_idxes = []
        for i, w in enumerate(sent):
            if w in en_uncommon:
                sent[i] = '<unk>'
                del en_uncommon[w]
            elif w not in en_vocab:
                en_vocab[w] = len(en_vocab)
            en_idxes.append(en_vocab[sent[i]])
        en_inputs.append(en_idxes)
    del train_en

    for sent in dev_en:
        en_idxes = []
        for i, w in enumerate(sent):
            if w in en_uncommon:
                sent[i] = '<unk>'
                del en_uncommon[w]
            elif w not in en_vocab:
                en_vocab[w] = len(en_vocab)
            en_idxes.append(en_vocab[sent[i]])
        en_val.append(en_idxes)
    del dev_en

    for sent in train_de:
        de_idxes = []
        for i, w in enumerate(sent):
            if w in de_uncommon:
                sent[i] = '<unk>'
                del de_uncommon[w]
            elif w not in de_vocab:
                de_vocab[w] = len(de_vocab)
            de_idxes.append(de_vocab[sent[i]])
        de_inputs.append(de_idxes)
    del train_de

    for sent in dev_de:
        de_idxes = []
        for i, w in enumerate(sent):
            if w in de_uncommon:
                sent[i] = '<unk>'
                del de_uncommon[w]
            elif w not in de_vocab:
                de_vocab[w] = len(de_vocab)
            de_idxes.append(de_vocab[sent[i]])
        de_val.append(de_idxes)
    del dev_de

using BPE data, skipping this cell


In [13]:
# vocab={}
# vocab['<pad>'] = 0
# vocab['<unk>'] = 1 
# vocab['<sos>'] = 2
# vocab['<eos>'] = 3

# uncommon = {'then':0, 'thing':1}

# sample = ['<sos>', 'the', 'brown', 'cow', 'then', 'jumped', '<eos>']

# sidx = []

# for i, w in enumerate(sample):
#     if w in uncommon:
#         sample[i] = '<unk>'
#         del uncommon[w]
#     elif w not in vocab:
#         vocab[w] = len(vocab)
#     sidx.append(vocab[sample[i]])  
    
# print(vocab)
# print(uncommon)
# print(sample)
# print(sidx)

## Print examples of words that are infrequently used

In [32]:
print(len(en_uncommon), "of the", len(en_freq), "english words are only used once")
print(len(de_uncommon), "of the", len(de_freq), "german words are only used once")

en_idx_to_word = dict((v,k) for k,v in en_vocab.items())
de_idx_to_word = dict((v,k) for k,v in de_vocab.items())

for i in range(10):
    idx = en_uncommon[random.randint(0,(len(en_uncommon)))]
    print(en_idx_to_word[idx])
    
for i in range(10):
    idx = de_uncommon[random.randint(0,(len(de_uncommon)))]
    print(de_idx_to_word[idx])

23267 of the 58510 english words are only used once
71615 of the 129363 german words are only used once
cmos
super-cooperators
dnc
weighting
leat
herblock
you.
partible
trifling
cycads
kommunikationsgeräte
endozytose
rosabeth
unerschrockene
falschlaufenden
konsummiert
steuersätze
ghul
wirtschaftskräfte
verteilungsseite


## Remove infrequent words from the dictionary

In [47]:
# en_idx_to_word = dict((v,k) for k,v in en_vocab.items())
# de_idx_to_word = dict((v,k) for k,v in de_vocab.items())       

# print("english training")
# for sent in en_inputs:
#     for i, idx in enumerate(sent):
#         if idx in en_uncommon:
#             sent[i] = 1
#             del en_idx_to_word[idx]
        
# print("english validation")
# for sent in en_val:
#     for i, idx in enumerate(sent):
#         if idx in en_uncommon:
#             sent[i] = 1
#             del en_idx_to_word[idx]

# print("german training")
# for sent in de_inputs:
#     for i, idx in enumerate(sent):
#         if idx in de_uncommon:
#             sent[i] = 1
#             del de_idx_to_word[idx]
        
# print("german validation")
# for sent in de_val:
#     for i, idx in enumerate(sent):
#         if idx in de_uncommon:
#             sent[i] = 1
#             del de_idx_to_word[idx]            


# ========== Warning ==========

## The following code cell sorts the train and validation sets by the length, (smallest to largest)

In [10]:
inplengths = [len(x) for x in de_inputs]
vallengths = [len(x) for x in de_val]

sorted_indices_inp = np.argsort(inplengths)
sorted_indices_val = np.argsort(vallengths)

de_inputs_sorted = []
en_inputs_sorted = []
de_val_sorted = []
en_val_sorted = []

for s in sorted_indices_inp:
    de_inputs_sorted.append(de_inputs[s])
    en_inputs_sorted.append(en_inputs[s])

for s in sorted_indices_val:
    de_val_sorted.append(de_val[s])
    en_val_sorted.append(en_val[s])

de_inputs = de_inputs_sorted
en_inputs = en_inputs_sorted
de_val = de_val_sorted
en_val = en_val_sorted

In [11]:
print("longest sentence", len(de_inputs[-1]))

longest sentence 768


## Check to see if the sorted data is correct.


**en_inputs[112862] and de_inputs[112862] should give:**

- <sos> i need your surname . symmetrical objects generally -- spell it for me . <eos> 
    
- <sos> ich brauche ihren nachnamen . symmetrische objekte haben grundsätzlich – bitte buchstabieren sie ihn für mich . <eos> 
    
**en_val[3021] and de_val[3021] should give:**

- <sos> but where would be the nearest aed to help this patient ? <eos> 
    
- <sos> aber wo wäre der nächste aed , um diesem patienten zu helfen ? <eos>     
    

In [10]:
# DEPRECATED EXAMPLE

# en_idx_to_word = dict((v,k) for k,v in en_vocab.items())
# de_idx_to_word = dict((v,k) for k,v in de_vocab.items())

# for i in en_inputs[112862]:
#     print(en_idx_to_word[i], end=' ')

# print("")

# for j in de_inputs[112862]:
#     print(de_idx_to_word[j], end=' ')
    
# print("")
    
# for i in en_val[3021]:
#     print(en_idx_to_word[i], end=' ')

# print("")

# for j in de_val[3021]:
#     print(de_idx_to_word[j], end=' ')
    

# ========== Warning ==========

## The following code cell adds padding to all of the sentences in both the training and validation datasets

In [11]:
# DEPRECATED METHOD

# for i, sentence in enumerate(en_inputs):
#     diff = max_length_en - len(sentence)
#     if diff == 0:
#         continue
#     pad_array = [0]*diff
#     sentence = sentence + pad_array
#     en_inputs[i] = sentence

# for i, sentence in enumerate(de_inputs):
#     diff = max_length_de - len(sentence)
#     if diff == 0:
#         continue
#     pad_array = [0]*diff
#     sentence = sentence + pad_array
#     de_inputs[i] = sentence
    
# for i, sentence in enumerate(en_val):
#     diff = max_length_en_dev - len(sentence)
#     if diff == 0:
#         continue
#     pad_array = [0]*diff
#     sentence = sentence + pad_array
#     en_val[i] = sentence

# for i, sentence in enumerate(de_val):
#     diff = max_length_de_dev - len(sentence)
#     if diff == 0:
#         continue
#     pad_array = [0]*diff
#     sentence = sentence + pad_array
#     de_val[i] = sentence

# ========== Warning ==========

## The following code cell shrinks the number of training sentences from 196k to 50k

In [12]:
# DEPRECATED METHOD
# 
# en_inputs = en_inputs[:50000]
# de_inputs = de_inputs[:50000]
# print(len(en_inputs))

## The following block saves the data that has been processed

In [14]:
if usingBPE:
    print("using BPE data")
    
    idx_to_word = dict((v,k) for k,v in vocab.items())

    en_iwslt = {}
    de_iwslt = {}

    en_iwslt['idx2word'] = idx_to_word  
    de_iwslt['idx2word'] = idx_to_word

    en_iwslt['train'] = en_inputs
    de_iwslt['train'] = de_inputs

    en_iwslt['dev'] = en_val
    de_iwslt['dev'] = de_val
    
    de_iwslt['test'] = de_test

    with open('data/processed/english_bpe_40.pickle', 'wb') as handle:
        pickle.dump(en_iwslt, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open('data/processed/german_bpe_40.pickle', 'wb') as handle:
        pickle.dump(de_iwslt, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print("German vocab length", len(de_iwslt['idx2word']))
    print("English vocab length", len(en_iwslt['idx2word']))

else:
    en_idx_to_word = dict((v,k) for k,v in en_vocab.items())
    de_idx_to_word = dict((v,k) for k,v in de_vocab.items())

    en_iwslt = {}
    de_iwslt = {}

    en_iwslt['idx2word'] = en_idx_to_word  
    de_iwslt['idx2word'] = de_idx_to_word

    en_iwslt['train'] = en_inputs
    de_iwslt['train'] = de_inputs

    en_iwslt['dev'] = en_val
    de_iwslt['dev'] = de_val
    
    de_iwslt['test'] = de_test

    with open('data/processed/english_unk_sorted_100k.pickle', 'wb') as handle:
        pickle.dump(en_iwslt, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open('data/processed/german_unk_sorted_100k.pickle', 'wb') as handle:
        pickle.dump(de_iwslt, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print("German vocab length", len(de_iwslt['idx2word']))
    print("English vocab length", len(en_iwslt['idx2word']))

using BPE data
German vocab length 12388
English vocab length 12388


## Examples of how to access some of the data

In [14]:
print(de_val[44])
print(len(en_vocab))
print(len(de_vocab))
print(en_vocab['like'])

for i in range(1): 
    for j in range(len(de_val[i])):
        print(de_idx_to_word[de_val[i][j]], end=" ")
    print("\n")


[2, 13736, 1101, 12731, 5770, 13990, 14, 3]
10386
14294
233
<sos> danke . <eos> 



## Check to make sure that the data has been saved correctly (make sure that the filenames match). If you get two Trues, the test is passed.

In [49]:
with open('data/processed/english_bpe.pickle', 'rb') as handle:
    english = pickle.load(handle)
    
with open('data/processed/german_bpe.pickle', 'rb') as handle:
    german = pickle.load(handle)
    
print(en_iwslt == english)
print(de_iwslt == german)

True
True


In [13]:
print(german['train'][0])

[2, 865, 3]


In [14]:
de_inputs[0]

[2, 865, 3]